In [1]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Markdown
%matplotlib inline

# Maze Generation

    =====================================================
    GENERATE_MAZE
        input:
            n_cells (int):           width of maze
            
        output:
            maze ndarray(int,int):  the resulting maze
    =====================================================
        
Creates a square grid of cells with size equal to `n_cells` and selects the upper left cell as the intial cell in a grid labeled `visited`. Generates a maze by randomly removing walls between the cells and adding the connected cells neighbours to the `visited` grid until all cells have been reached by the initial cell.

In [2]:
def generate_maze(n_cells):
    # Generate initial grid
    cells = np.zeros((n_cells * 2 + 1, n_cells * 2 + 1))
    cells[::2] = 1
    cells.T[::2] = 1

    # Create maze recursively
    visited = np.zeros((n_cells,n_cells))
    candidates = [[0,0]]
    while candidates != []:
        np.random.shuffle(candidates)
        if candidates == []:
            break
        candidate = np.array(candidates.pop())
        directions = [0,1,2,3]
        np.random.shuffle(directions)

        for direction in directions:
            # Get the neighbour coordinate
            direction = np.array(np.round([np.sin(direction * np.pi / 2), np.cos(direction * np.pi / 2)]), dtype=int)
            neighbour = np.array([candidate[0] + direction[0], candidate[1] + direction[1]], dtype=int)
            
            # Check if the neighbour is within bounds
            if neighbour[0] >= n_cells or neighbour[0] < 0 \
                    or neighbour[1] >= n_cells or neighbour[1] < 0 \
                    or visited[neighbour[0]][neighbour[1]] == 1: continue

            candidates += [candidate, neighbour]
            visited[neighbour[0], neighbour[1]] = 1
            candidate = np.asarray(candidate) * 2 + 1
            candidate = [candidate[0] + direction[0], candidate[1] + direction[1]]
            cells[candidate[0]][candidate[1]] = 0
            break
            
    # Trim the border cells; return
    return cells[1:-1][:,1:-1]

    =====================================================
    GENERATE_WIDE_MAZE
        input:
            n_cells (int):           width of maze
            cell_width (int):        width of the individual cells of the maze
            wall_width (int):        width of the walls within the maze
            
        output:
            maze ndarray(int,int):  the resulting maze
    =====================================================
        
Creates a square grid with size equal to `n_cells` and naively stretches the maze so that the individual cells of the maze are larger than the initial width of 1. `TODO:` implement the wall width as well.

In [3]:
# Extend a maze so that the cells are wider
def generate_wide_maze(n_cells, cell_width=1, wall_width=1):
    maze = generate_maze(n_cells)
    
    # Cell widths
    nm = []
    for j in range(len(maze[0])):
        m = []
        for i in range(len(maze[0])):
            if i % 2 == 0:
                for x in range(cell_width-1):
                    m.append(maze[j][i])
            m.append(maze[j][i])
        nm.append(m)
    nnm = []
    nm = np.asarray(nm).T.tolist()
    for i in range(len(nm)):
        m = []
        for j in range(len(nm[0])):
            if j % 2 == 0:
                for x in range(cell_width-1):
                    m.append(nm[i][j])
            m.append(nm[i][j])
        nnm.append(m)
    return np.asarray(nnm, dtype=int)

# A* Algorithm

    =====================================================
    A_STAR
        input:
            cells:                        the cells containing the maze
            start [int,int]:              x,y coordinates for the starting position
            end  [int,int]:               x,y coordinates for the end position
            h     [[int,int], [int,int]]: function used to calculate the distance between two points
        output: 
            path:                         The cells that form the path from `start` to `end`
    =====================================================


### Wikipedia Pseudocode
    reconstruct(source, current):
        total_path = {current}
        while current in source.keys:
            current = source[current]
            total_path.prepend(current)
        return total_path
        
    a*(start, goal, h)
    openset = {start}
    source = {}
    
    gscore = {default = inf}
    gscore[start] = 0
    
    fscore = {default = inf}
    fscore[start] = h[start]
    
    while openset != {}
        current = lowest fscore in openset
        if current = goal:
            return reconstruct_path(source, current)
        openset.remove(current)
        for each neighbour in current:
            tentative_gscore = gscore[current] + distance(current, neighbour)
            
            if tentative_gscore < gscore[neighbour]
                source[neighbour] = current
                gscore[neighbour] = tentative_gscore
                fscore[neighbour] = tentative_gscore + h(neighbour)
                
                if neighbor not in openset:
                    openset.add(neighbour)
    return None
            
        